In [ ]:
from astropy.time import Time, TimeDelta
import astropy.units as u
import plotly.express as px

from lsst_efd_client import EfdClient

In [ ]:
# csc = "ATHexapod"
# csc = "Test:42"
csc = "LEDProjector"
topic_name = "logevent_heartbeat"
expected_rate = 1
use_kafka = True
efd_name = "tucson_teststand_efd"
# start_time_str = "2023-11-13T23:00:00"
# end_time_str = "2023-11-13T23:15:00"
start_time_str = "2024-02-13T19:15:00"
end_time_str = "2024-02-13T19:22:00"

In [ ]:
client = EfdClient(efd_name)

In [ ]:
if ":" in csc:
    values = csc.split(":")
    csc_name = values[0]
    csc_index = int(values[1])
else:
    csc_name = csc
    csc_index = None
plot_title = f"{csc} {topic_name}"

In [ ]:
start_time = Time(start_time_str, scale='utc')
end_time = Time(end_time_str, scale='utc')

columns = ["private_sndStamp", "private_rcvStamp", "private_seqNum"]
if use_kafka:
    columns.append("private_kafkaStamp")
df = await client.select_time_series(f"lsst.sal.{csc_name}.{topic_name}",
                                     columns,
                                     start_time,
                                     end_time,
                                     index=csc_index)

In [ ]:
for column in columns:
    df[f"{column}_diff"] = df[column].diff()

In [ ]:
fig = px.scatter(df, x="private_sndStamp", y="private_kafkaStamp")
fig.show()